In [18]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
%matplotlib inline


In [19]:
g = torch.Generator().manual_seed(2147483647)

class Linear:
    def __init__(self , fan_in , fan_out , bias=True):
        self.weight = torch.randn((fan_in , fan_out) , generator=g) / fan_in ** 0.5
        self.bias = torch.zeros(fan_out) if bias else None
    
    def __call__(self, x):
        self.out = x @ self.weight 
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])


class BatchNorm1d:
    def __init__(self , dim , eps=1e-5 , momentum = 0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True

        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self , x):
        if self.training:
            xmean = x.mean(0 , keepdim = True)
            xvar = x.var(0 , keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta

        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out 

    def parameters(self):
        return [self.gamma , self.beta]
    
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []


In [20]:
words = open("names.txt" , 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i , s in enumerate(chars)}
stoi['.'] = 0
itos = {i : s for s , i in stoi.items()}
vocab_size = len(itos)

block_size = 3
def build_dataset(words):
    X , Y = [] , []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context) , '--->' , itos[ix])
            context = context[1 :] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape , Y.shape) 
    return X , Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr , Ytr = build_dataset(words[:n1])     #train data
Xdev , Ydev = build_dataset(words[n1:n2]) #dev data
Xte , Yte = build_dataset(words[n2:])     #test data

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [24]:
n_embd = 10
n_hidden = 100


C = torch.randn((vocab_size , n_embd) , generator=g)

layers = [
    Linear(n_embd * block_size , n_hidden),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden , n_hidden),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden , n_hidden),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden , n_hidden),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden , n_hidden),
    Tanh(),
    Linear(n_hidden , vocab_size),
    BatchNorm1d(vocab_size),
]

with torch.no_grad():
    layers[-1].gamma *= 0.1
    for layer in layers[:-1]:
        if isinstance(layer , Linear):
            #为我们在初始化时，使其标准差为1，但tanh会压缩，所以需要进行扩张而对抗压缩
            #当这个值过小，会使结果趋近于0，即所有神经元保持在活跃状态 
            #当这个值过大，会使结果趋近于两端，即大多数神经元处于死亡状态（tanh在两端梯度很小，很难被更新）
            #我们希望使各个层的梯度基本一致，几乎在同时达到收敛状态，而不是有些层收敛快
            #有些层收敛慢
        
            layer.weight *= 5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))

for p in parameters:
    p.requires_grad = True

47351


In [25]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    ix = torch.randint(0 , Xtr.shape[0] , (batch_size , ) , generator=g)
    Xb , Yb = Xtr[ix] , Ytr[ix]

    emb = C[Xb]
    x = emb.view(emb.shape[0] , -1)

    #forward
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x , Yb)

    #backward
    for layer in layers:
        layer.out.retain_grad()
    
    for p in parameters:
        p.grad = None

    loss.backward()

    #update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    #track stats
    if (i + 1) % 10000 == 0:
        print(f'{i + 1:7d}/{max_steps:7d}: {loss.item():.4f}')
    elif i == 0:
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())
    break

      0/ 200000: 3.3206
